# Exploring Toronto, Canada: The battle of neighborhoods!

## Table of contents:
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

Toronto is the capital city of the Canadian province of Ontario, and the most populous city in Canada. Toronto is an international centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world. It showcases diversity with strengths in technology, education, fashion, food services, and tourism. 

Let's explore the neighbourhoods around Toronto using data science! 

In this data science capstone project, we will explore Toronto, and find out the most popular places (such as museums, parks, café, restaurants, etc.), to help guide the tourists, investors and general public interested in visiting Toronto.

We will use the data science skills acquired as part of the IBM Data Science Professional Certification (by Coursera), and generate a few most popular neighbourhoods with information about interesting venues to visit. The details about each venue will then be suggested for the best possible exploration of Toronto.

## Data <a name="data"></a>

Based on the problem definition, following factors will contribute in deciding which neighborhood is popular:
* number of museums and attractions in the neighborhood.
* number of café, shops and restaurants in the neighborhood.
* whether the neighborhood is close to the city center.


Following data sources will be needed to extract the required information:

* **Wikipedia**, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, to obtain the information related to the neighbourhoods in Toronto.

* **Foursquare API** to obtain the location/venue data (i.e. number of museums, attractions, café, restaurants, etc.)
* **BeautifulSoup** to parse the area information from a HTML table.
* **Geocoder** to obtain the area coordinates.
* **Folium** to visualize the maps.

### Neighborhood Candidates

So let's get started! <br>
Let's first install and import the required libraries, and then proceed with finding the latitude and longitude coordinates for the centroids of our candidate neighborhoods around Toronto.

Import the libraries:

In [1]:
!pip install geocoder
!pip install folium
!pip install geopy

You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Libraries Imported!")

Libraries Imported!


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data)
table_data=wiki_data.find('div', class_='mw-parser-output')
table = table_data.table.tbody

In [4]:
columns = ['PostalCode', 'Borough', 'Neighbourhood']

data = dict({key:[]*len(columns) for key in columns})

for row in table.find_all('tr'):
    for i,column in zip(row.find_all('td'),columns):
        i = i.text
        i = i.replace('\n', '')
        data[column].append(i)

df = pd.DataFrame.from_dict(data=data)[columns]

In [5]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighbourhood != empty)]

## Cleaning the dataframe: 
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. More than one neighborhood can exist in one postal code area. These will be combined into one with the neighborhoods separated with a comma. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop = True)

In [7]:
pc, brgh, n = [], [], []
for postcode, borough, neigh in zip(df['PostalCode'], df['Borough'], df['Neighbourhood']):
    pc.append(postcode)
    brgh.append(borough)
    if neigh == 'Not assigned':
        n.append(borough)
    else:
        n.append(neigh)

df = pd.DataFrame({'PostalCode': pc, 'Borough': brgh, 'Neighbourhood':n})[columns]
print(df.shape)

(20, 3)


In [8]:
postcodes = df['PostalCode'].values
boroughs = df['Borough'].values
neighs = df['Neighbourhood'].values

#create a dictionary with keys as Postcode and Borough
dic = dict({(key1,key2): [] for key1, key2 in zip(postcodes, boroughs)})
print('Number of keys: ', len(dic.keys()))

for postcode, borough, neigh in zip(postcodes,boroughs, neighs):
    key = (postcode, borough)
    dic[key].append(neigh)

df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighbourhood'])
for key, value in dic.items():
    postcode, borough, neig = key[0], key[1], value
    neig = ', '.join(neig)
    df = df.append({'Postal Code': postcode,
                     'Borough': borough,
                     'Neighbourhood': neig}, ignore_index = True)
print('Shape: ', df.shape)

Number of keys:  20
Shape:  (20, 3)


In [9]:
# save
#df.to_csv('toronto.csv', index=False)

In [10]:
df = pd.read_csv('toronto.csv') 
print(df.shape)
df.head()

(289, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.709020000000066, -79.36348999999996]

In [12]:
# Retrieve Postal Code Coordinates
postal_codes = df['Postcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

# Add columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [13]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for borough, neighborhood, lat, lng in zip(df['Borough'], df['Neighbourhood'], df['Latitude'], df['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
print('Total number of Boroughs = ', len(df['Borough'].unique()))

Total number of Boroughs =  12


In [16]:
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(74, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.65512,-79.36264
1,M5A,Downtown Toronto,Regent Park,43.65512,-79.36264
2,M5B,Downtown Toronto,Ryerson,43.65739,-79.37804
3,M5B,Downtown Toronto,Garden District,43.65739,-79.37804
4,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587


In [17]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toronto)  
map_toronto

## Methodology <a name="methodology"></a>

After successfully extracting the information about Toronto and naighbourhoods from its Wikipedia page, and visualizing it on the city map, we will now use the Foursquare API to explore venues around the area. 

In [18]:
CLIENT_ID = '****' # your Foursquare ID
CLIENT_SECRET = '****' # your Foursquare Secret
VERSION = '20210623'
LIMIT = 100 # A default Foursquare API limit value

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [21]:
df = toronto_data
toronto_venues = getNearbyVenues(names=df['Neighbourhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [24]:
print(toronto_venues.shape)
toronto_venues.head(20)

(3376, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,Harbourfront,43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
2,Harbourfront,43.65512,-79.36264,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
3,Harbourfront,43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space
4,Harbourfront,43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
5,Harbourfront,43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa
6,Harbourfront,43.65512,-79.36264,Rooster Coffee,43.651900,-79.365609,Coffee Shop
7,Harbourfront,43.65512,-79.36264,Sumach Espresso,43.658135,-79.359515,Coffee Shop
8,Harbourfront,43.65512,-79.36264,Subway,43.655701,-79.364644,Sandwich Place
9,Harbourfront,43.65512,-79.36264,Mangia and Bevi Resto-Bar,43.652250,-79.366355,Italian Restaurant


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 215 uniques categories.


In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [29]:
toronto_onehot.shape

(3376, 215)

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(20)

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.0,0.010000,0.000000,0.040000,0.000000,0.000000,0.000000,0.010000,0.010000,0.030000,0.010000,0.00,0.010000,0.000000,0.000000,0.0,0.000000,0.0,0.010000,0.0,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.0,0.000000,0.000000,0.080000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.130000,0.0,0.000000,0.0,0.01,0.000000,0.000000,0.020000,0.0,0.000000,0.010000,0.00,0.0,0.0,0.000000,0.030000,0.010000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010000,0.000000,0.000000,0.020000,0.000000,0.0,0.000000,0.01,0.000000,0.010000,0.000000,0.030000,0.010000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.060000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.

In [32]:
toronto_grouped.shape

(73, 215)

In [33]:
num_top_venues = 5
for neigh in toronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0       Coffee Shop  0.13
1              Café  0.08
2             Hotel  0.06
3    Sandwich Place  0.06
4  Asian Restaurant  0.04


----Bathurst Quay----
                  venue  freq
0    Italian Restaurant  0.08
1           Coffee Shop  0.06
2        Sandwich Place  0.05
3                  Park  0.05
4  Gym / Fitness Center  0.04


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.08
1              Bakery  0.06
2      Sandwich Place  0.06
3  Seafood Restaurant  0.04
4      Farmers Market  0.04


----Brockton----
            venue  freq
0             Bar  0.09
1  Sandwich Place  0.06
2     Coffee Shop  0.05
3      Restaurant  0.05
4     Yoga Studio  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.11
1         Café  0.06
2        Hotel  0.04
3   Restaurant  0.03
4          Gym  0.03


----CN Tower----
                  venue  freq
0    Italian Restaurant  0.08
1    

## Most popular venues around the neighborhood

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(73, 11)

## Clustering the Neighborhoods using K-means

In [65]:
from sklearn.cluster import KMeans
import sklearn.cluster.k_means_

In [66]:
# Using K-Means to cluster neighborhood into 5 clusters
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[0 0 0 0 0 0 0 0 0 0]
73


In [67]:
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.65512,-79.36264
1,M5A,Downtown Toronto,Regent Park,43.65512,-79.36264
2,M5B,Downtown Toronto,Ryerson,43.65739,-79.37804
3,M5B,Downtown Toronto,Garden District,43.65739,-79.37804
4,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
5,M4E,East Toronto,The Beaches,43.67709,-79.29547
6,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
7,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
8,M6G,Downtown Toronto,Christie,43.66869,-79.42071
9,M5H,Downtown Toronto,Adelaide,43.64970,-79.38258


In [68]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.65512,-79.36264,0,Coffee Shop,Pub,Distribution Center,Italian Restaurant,Discount Store,Electronics Store,Thai Restaurant,Restaurant,Bakery,Spa
1,M5A,Downtown Toronto,Regent Park,43.65512,-79.36264,0,Coffee Shop,Pub,Distribution Center,Italian Restaurant,Discount Store,Electronics Store,Thai Restaurant,Restaurant,Bakery,Spa
2,M5B,Downtown Toronto,Ryerson,43.65739,-79.37804,0,Coffee Shop,Sandwich Place,Hotel,Café,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Pizza Place,Bar
3,M5B,Downtown Toronto,Garden District,43.65739,-79.37804,0,Coffee Shop,Sandwich Place,Hotel,Café,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Pizza Place,Bar
4,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Cocktail Bar,Clothing Store,Cosmetics Shop,Diner,Japanese Restaurant,Bakery
5,M4E,East Toronto,The Beaches,43.67709,-79.29547,0,Health Food Store,Coffee Shop,Pub,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
6,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,0,Cocktail Bar,Sandwich Place,Bakery,Farmers Market,Coffee Shop,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Butcher,Spa
7,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493,0,Coffee Shop,Clothing Store,Pizza Place,Sandwich Place,Sushi Restaurant,Bank,Cosmetics Shop,Restaurant,Plaza,Japanese Restaurant
8,M6G,Downtown Toronto,Christie,43.66869,-79.42071,0,Café,Grocery Store,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Escape Room,Electronics Store,Elementary School,Wings Joint
9,M5H,Downtown Toronto,Adelaide,43.64970,-79.38258,0,Coffee Shop,Café,Sandwich Place,Hotel,Japanese Restaurant,Restaurant,Asian Restaurant,Deli / Bodega,Sushi Restaurant,Gym


## Visualize the cluster!

In [70]:
# Create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters

## Results and Discussion <a name="results"></a>

As a result, the neighborhoods around Toronto were segmented into 5 clusters using the K-means clustering approach (unsupervised machine learning), where each cluster represent a group of similar neighborhods. In the following data frames, one can observe the details about the venues in each of the 5 clusters. 

**First cluster** (see Cluster Label 0) represents the neighborhoods sharing similar venues, such as coffee shops, pubs, hotels, café, restaurants, shops, etc. being the most popular. This is a nice neighborhood from both living and tourist point of view, and is quite popular amongst the locals and the visitors. There are many coffee shpps, pubs and restaurants, with a wide variety of cuisines. It is near the downtown, and there are many interesting shops to buy clothes, groceries, etc. Also, there are hotels nearby, making it a perfect neighborhood for tourists to stay and explore.

**Second cluster** (see Cluster Label 1) is focused around central Toronto, and includes venues of interest such rail station, coffee shop, wings joint, dogs running area, and farmers market, etc. 

**Third cluster** (see Cluster Label 2) highlights child friendly neighbourhood, with parks, playgrounds and bike trails being the most common venues. 

**Fourth cluster** (see Cluster Label 3) is dominated by facilities such as home service, restaurants and farms. It is relatively a quiet neighbourhood in the central Toronto area.

**Fifth cluster** (see Cluster Label 4) consists of neighborhoods popular for parks, gyms, trail, restaurants, etc. and is popular amongst the locals. There is also an escape room in the neighbourhood. 

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0]+[1]+list(range(5, toronto_merged.shape[1]))]]

,Postcode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,0,Coffee Shop,Pub,Distribution Center,Italian Restaurant,Discount Store,Electronics Store,Thai Restaurant,Restaurant,Bakery,Spa
1,M5A,Downtown Toronto,0,Coffee Shop,Pub,Distribution Center,Italian Restaurant,Discount Store,Electronics Store,Thai Restaurant,Restaurant,Bakery,Spa
2,M5B,Downtown Toronto,0,Coffee Shop,Sandwich Place,Hotel,Café,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Pizza Place,Bar
3,M5B,Downtown Toronto,0,Coffee Shop,Sandwich Place,Hotel,Café,Clothing Store,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Pizza Place,Bar
4,M5C,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Cocktail Bar,Clothing Store,Cosmetics Shop,Diner,Japanese Restaurant,Bakery
5,M4E,East Toronto,0,Health Food Store,Coffee Shop,Pub,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
6,M5E,Downtown Toronto,0,Cocktail Bar,Sandwich Place,Bakery,Farmers Market,Coffee Shop,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,Butcher,Spa
7,M5G,Downtown Toronto,0,Coffee Shop,Clothing Store,Pizza Place,Sandwich Place,Sushi Restaurant,Bank,Cosmetics Shop,Restaurant,Plaza,Japanese Restaurant
8,M6G,Downtown Toronto,0,Café,Grocery Store,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Escape Room,Electronics Store,Elementary School,Wings Joint
9,M5H,Downtown Toronto,0,Coffee Shop,Café,Sandwich Place,Hotel,Japanese Restaurant,Restaurant,Asian Restaurant,Deli / Bodega,Sushi Restaurant,Gym


In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0]+[1]+list(range(5, toronto_merged.shape[1]))]]

,Postcode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,M4V,Central Toronto,1,Light Rail Station,Coffee Shop,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
55,M4V,Central Toronto,1,Light Rail Station,Coffee Shop,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
56,M4V,Central Toronto,1,Light Rail Station,Coffee Shop,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
57,M4V,Central Toronto,1,Light Rail Station,Coffee Shop,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
58,M4V,Central Toronto,1,Light Rail Station,Coffee Shop,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0]+[1]+list(range(5, toronto_merged.shape[1]))]]

,Postcode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,M5P,Central Toronto,2,Park,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Elementary School
35,M5P,Central Toronto,2,Park,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Elementary School
66,M4W,Downtown Toronto,2,Park,Playground,Bike Trail,Wings Joint,Dog Run,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0]+[1]+list(range(5, toronto_merged.shape[1]))]]

,Postcode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M5N,Central Toronto,3,Home Service,Wings Joint,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0]+[1]+list(range(5, toronto_merged.shape[1]))]]

,Postcode,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,M4T,Central Toronto,4,Park,Gym,Trail,Restaurant,Wings Joint,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
50,M4T,Central Toronto,4,Park,Gym,Trail,Restaurant,Wings Joint,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room


## Conclusion <a name="conclusion"></a>

The purpose of this project was to explore Toronto and find out the most popular venues around the neighbourhood.
Using the Foursquare API, we gathered the location data about the popular venues around Toronto. Using unsupervised machine learning approach, K-means clustering, we clustered the locations to generate the major areas of interest containing the most popular places to explore. 

Final decision on the best neighborhood to visit will be made by the tourists, visitors or investors, based on their personal preferences about the specific characteristics they like in a neighborhood in every recommended zone (or cluster).

For example: 

* If you are a tourist visiting Toronto, the neighborhoods in the first cluster are a must visit!

* If you are an investor, interested in the restaurant business,  do check the neighborhoods and venues in the first, second and fifth cluster.

* If you want to visit trails, check out the third and fifth cluster.

* Do check out the map above to explore the neighborhoods, and refer to the tables for information about the venues that interests you.

Thank you! <br>
This page will be further updated, as I am interested in leanring more about data science! <br>